In [ ]:
import itertools
import numpy as np
import healpy as hp
import sphere.distribution as sd

from matplotlib import cm
from matplotlib import pyplot as plt
%matplotlib inline


def sphe_to_cart(r, theta, phi):
    x = r*np.sin(theta)*np.cos(phi)    
    y = r*np.sin(theta)*np.sin(phi)
    z  = r*np.cos(theta)

    return x, y, z


def sphe_to_kent(theta, phi):                                                                                                                                                                                                                 
    """ shift x,y,z->x2,x3,x1 which is the coordinate system used in Kent 1982

    return np.array((x1,x2,x3), ...) which is the format read-in by kent_distribution
    """
    xyz = sphe_to_cart(1, theta, phi)
    xs = np.asarray((xyz[2], xyz[0], xyz[1])).T
    return xs


def kent_map(kentobj, nside):
    """Returns a map of the kent distribution's pdf in the specified
    coordinates                                                                                                                                                                                                                               
    """                                                                                                                                                                                                                                       
    npix = hp.nside2npix(nside)
    zen, azi = hp.pix2ang(nside, np.arange(npix))
    return kentobj.pdf(sphe_to_kent(zen, azi))

In [ ]:
etas = [-1,-0.8,0,0.8,1]
betas = [0.1,1,5,10]
kappas = [0.1,1,5,10]

curr_kappa = 0
curr_ax = 0
for kappa, beta, eta in itertools.product(kappas, betas, etas):
    kmap = kent_map(sd.kent(np.pi/2,0,0,kappa,beta,eta),64)
    
    vmap = cm.plasma
    vmap.set_under('w')
    vmap.set_bad('w')
    if kappa != curr_kappa:
        if curr_ax:
            plt.savefig('fig/k{}.png'.format(curr_kappa),bbox_inches='tight')
        fig, axes = plt.subplots(len(betas), len(etas), figsize=(7*len(etas),5*len(betas)), dpi=300)
        flat_axes = axes.flatten()
        curr_kappa = kappa
        curr_ax = 0
    plt.sca(flat_axes[curr_ax])
    hp.mollview(kmap, title=r'$\kappa={}$, $\beta={}$, $\eta={}$'.format(kappa,beta,eta),
                min=0, max=np.round(np.nanmax(kmap),2),
                cmap=vmap,hold=True,cbar=True,xsize=1600)
    hp.graticule()
    curr_ax+=1
plt.savefig('fig/k{}.png'.format(kappa),bbox_inches='tight')